*MERGING DATA INTO ONE DATASET*

In [1]:
#IMPORTING MAIN PACKAGES
import pandas as pd 
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
#READING THE 3 FILES
players_stats = pd.read_pickle("player_stats.pkl")
teams_stats= pd.read_pickle("teams_stats.pkl")
players_awards = pd.read_pickle("players_awards.pkl")


In [46]:
#RENAME COLUMN YERA FROM teams_stats
teams_stats.rename(columns={'YEAR':'SEASON_ID'},inplace=True)

In [47]:
#MERGE players_stats and teams_stats into players_teams
players_teams = players_stats.merge(teams_stats.loc[:,['TEAM_ID','SEASON_ID','WIN_PCT', 'CONF_RANK', 
 'DIV_RANK']],how='inner',on=['TEAM_ID','SEASON_ID'],)

In [48]:
players_teams

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,WIN_PCT,CONF_RANK,DIV_RANK
0,51,1990-91,00,1610612743,DEN,22.0,67,19,1505.0,417,...,121,206,55,4,110,149,942,0.244,14,7
1,422,1990-91,00,1610612743,DEN,26.0,21,4,217.0,29,...,21,77,14,2,27,22,79,0.244,14,7
2,193,1990-91,00,1610612743,DEN,24.0,3,0,21.0,2,...,5,0,1,0,0,6,10,0.244,14,7
3,371,1990-91,00,1610612743,DEN,23.0,17,0,279.0,56,...,88,16,16,9,18,44,128,0.244,14,7
4,51,1991-92,00,1610612743,DEN,23.0,81,11,1538.0,356,...,114,192,44,4,117,130,837,0.293,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14334,788,1987-88,00,1610612737,ATL,25.0,75,55,2091.0,356,...,547,28,68,42,138,240,871,0.610,3,2
14335,788,1989-90,00,1610612737,ATL,27.0,81,51,2273.0,418,...,645,57,63,47,144,259,1006,0.500,9,6
14336,788,1990-91,00,1610612737,ATL,28.0,80,80,2373.0,444,...,704,99,60,40,153,235,1051,0.524,6,4
14337,766,1986-87,00,1610612755,PHL,23.0,77,9,1612.0,259,...,156,155,93,19,128,169,680,0.549,5,2


In [49]:
# creating count of 'player of the week' & 'player of the month'  
week = players_awards[players_awards['DESCRIPTION']=='NBA Player of the Week'].groupby(['PERSON_ID','SEASON'])
month = players_awards[players_awards['DESCRIPTION']=='NBA Player of the Month'].groupby(['PERSON_ID','SEASON'])
player_of_week = []
player_of_month = []
rockie_of_month = []
for i in tqdm(range(len(players_teams))):

    player_id = players_teams.loc[i,'PLAYER_ID']
    season_id = players_teams.loc[i,'SEASON_ID']
    try:
        player_of_month.append(month.get_group((player_id,season_id))['PERSON_ID'].count())
    except:
        player_of_month.append(0)
    
    try:
        player_of_week.append(week.get_group((player_id,season_id))['PERSON_ID'].count())
    except:
        player_of_week.append(0)

100%|██████████| 14339/14339 [00:00<00:00, 17771.67it/s]


In [50]:
#ADDING THE AWARDS COLUMNS TO THE MERGED DATASET
players_teams['player_of_week'] = player_of_week
players_teams['player_of_month'] = player_of_month

In [51]:
#CREATING THE TARGET FEATURE `MVP` AND BUILDING AN NEW DATASET
mvp = players_awards[players_awards['DESCRIPTION']=='NBA Most Valuable Player'].rename(columns={"PERSON_ID":"PLAYER_ID","SEASON":"SEASON_ID","TYPE":"MVP"})

In [52]:
df= players_teams.merge(mvp.loc[:,["PLAYER_ID","SEASON_ID","MVP"]],how="left",on=["PLAYER_ID","SEASON_ID"])

In [53]:
df["MVP"] = df.MVP.map({"Award":1,np.NaN:0})

In [55]:
df

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,BLK,TOV,PF,PTS,WIN_PCT,CONF_RANK,DIV_RANK,player_of_week,player_of_month,MVP
0,51,1990-91,00,1610612743,DEN,22.0,67,19,1505.0,417,...,4,110,149,942,0.244,14,7,0,0,0
1,422,1990-91,00,1610612743,DEN,26.0,21,4,217.0,29,...,2,27,22,79,0.244,14,7,0,0,0
2,193,1990-91,00,1610612743,DEN,24.0,3,0,21.0,2,...,0,0,6,10,0.244,14,7,0,0,0
3,371,1990-91,00,1610612743,DEN,23.0,17,0,279.0,56,...,9,18,44,128,0.244,14,7,0,0,0
4,51,1991-92,00,1610612743,DEN,23.0,81,11,1538.0,356,...,4,117,130,837,0.293,11,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14334,788,1987-88,00,1610612737,ATL,25.0,75,55,2091.0,356,...,42,138,240,871,0.610,3,2,0,0,0
14335,788,1989-90,00,1610612737,ATL,27.0,81,51,2273.0,418,...,47,144,259,1006,0.500,9,6,0,0,0
14336,788,1990-91,00,1610612737,ATL,28.0,80,80,2373.0,444,...,40,153,235,1051,0.524,6,4,0,0,0
14337,766,1986-87,00,1610612755,PHL,23.0,77,9,1612.0,259,...,19,128,169,680,0.549,5,2,0,0,0


In [56]:
with open("df.pkl",'wb') as f:
    pickle.dump(df,f)